### Import modules

In [ ]:
import pandas as pd
import datetime as dt
import statsmodels.api as sm
import yfinance as yf
import statsmodels.api as sm
from scipy.stats import f
import plotly.express as px
import seaborn as sns
import numpy as np
import copy
import matplotlib.pyplot as plt
from matplotlib.ticker import FuncFormatter, MaxNLocator

from pypfopt.efficient_frontier import EfficientFrontier
from pypfopt import risk_models
from pypfopt import expected_returns
from pypfopt import plotting

#from zipline.api import order_percent, symbols
#from zipline.finance import commission

### Read F-F dataset

In [ ]:
factors = pd.read_csv("F-F_Research_Data_Factors_daily.CSV", sep=",")
factors["date"] = pd.to_datetime(factors["date"], format="%Y%m%d")
factors.set_index("date", inplace=True)
factors.replace([-99.99], 0, inplace = True)
factors.replace([-999], 0, inplace = True)
factors.dropna(inplace=True)
#factors["Mkt"] = factors["Mkt-RF"]+ff3factor["RF"]
factors = factors/100 #Get %returns
factors

### Calculate from daily returns an index level (basis = 100)

In [ ]:
# Calculate the cumulative daily returns
df_cum_daily_returns_ff = 100*(1 + factors).cumprod() # index level start = 100
#df_cum_daily_returns = df_cum_daily_returns.reset_index()
df_cum_daily_returns_ff

### Plot F-F dataset Mkt daily excess return

In [ ]:
fig = px.line(factors["Mkt-RF"])
fig.show()

### Plot Market excess returns index

In [ ]:
index_Mkt_ret = df_cum_daily_returns_ff["Mkt-RF"]
fig = px.line(index_Mkt_ret)
fig.show()

### Rebase on monthly returns: product of (1+daily return) in slice of months

In [ ]:
factors.index = pd.to_datetime(factors.index)
df_monthly_ff = factors.resample('M').agg(lambda x: (x + 1).prod() - 1)
df_monthly_ff

In [ ]:
df_monthly_ff.describe()

### Plot F-F monthly Mkt excess returns and factors

In [ ]:
fig = px.line(df_monthly_ff)
fig.show()

### Read F-F 49 industries dataset

In [ ]:
industries  = pd.read_csv("49_Industry_Portfolios_Daily.csv", sep=",")
industries ["date"] = pd.to_datetime(industries ["date"], format="%Y%m%d")

industries.set_index("date", inplace=True)
industries.replace([-99.99], 0, inplace = True)
industries.replace([-999], 0, inplace = True)
industries.dropna(inplace=True)
industries = industries/100 # turns into percentage
industries

### Plot F-F 49 industries dataset daily returns

In [ ]:
fig = px.line(industries)
fig.show()

In [ ]:
# August 1932 : Paper industry daily returns spiked
industries.describe()

### Plot F-F 49 industries dataset returns index (basis = 100)

In [ ]:
# Calculate the cumulative daily returns
df_industries_daily_index = 100*(1 + industries).cumprod() # index level start = 100
#df_cum_daily_returns = df_cum_daily_returns.reset_index()
df_industries_daily_index

### Plot F-F 49 industries daily returns index

In [ ]:
index_df_cum_daily_returns = df_industries_daily_index
fig = px.line(index_df_cum_daily_returns)
fig.show()

### Resample F-F 49 industries monthly returns

In [ ]:
industries.index = pd.to_datetime(industries.index)
df_industries_monthly_returns = industries.resample('1M').agg(lambda x: (x + 1).prod() - 1)
df_industries_monthly_returns

In [ ]:
df_industries_monthly_returns.describe()

### Plot F-F 49 industries monthly returns

In [ ]:
fig = px.line(df_industries_monthly_returns)
fig.show()

### Thinking of a sector rotation strategy : Find best and worst performing 3 industries for each period

In [ ]:
# Find best and worst performing 3 industries for each period

maxdf = df_industries_monthly_returns.apply(lambda x: pd.Series(x.nlargest(3).index.values), axis=1) # best three
maxdf.columns = ['Max1', 'Max2', 'Max3']
mindf = df_industries_monthly_returns.apply(lambda x: pd.Series(x.nsmallest(3).index.values), axis=1) # worst three
mindf.columns = ['Min1', 'Min2', 'Min3']

#print(df_industries_monthly_returns)
print(maxdf)
print(mindf)

### Define functions for portfolio sector rotation strategy 

In [ ]:
# We will long the best m stocks of the previous month 
# and rebalance on a monthly basis by removing the worst x stocks of the portfolio

def pflio_long(DF,m,x):
    
    # Returns cumulative portfolio return
    # DF = dataframe with monthly return info for all stocks
    # m = number of stock in the portfolio
    # x = number of underperforming stocks to be removed from portfolio monthly"""
    
    df = DF.copy()
    portfolio = []
    monthly_ret_long = [0]
    for i in range(1,len(df)):
        if len(portfolio) > 0:
            monthly_ret_long.append(df[portfolio].iloc[i,:].mean())
            bad_stocks = df[portfolio].iloc[i,:].sort_values(ascending=True)[:x].index.values.tolist()
            portfolio = [t for t in portfolio if t not in bad_stocks]
        fill = m - len(portfolio)
        new_picks = df.iloc[i,:].sort_values(ascending=False)[:fill].index.values.tolist()
        # We avoid to have two time the same stock in the portfolio
        new_picks = [t for t in new_picks if t not in portfolio] 
        portfolio = portfolio + new_picks
    monthly_ret_df_long = pd.DataFrame(np.array(monthly_ret_long),columns=["mon_ret"])
    return monthly_ret_df_long

In [ ]:
# We will short the worst m stocks of the previous month 
# and rebalance on a monthly basis by removing the best x stocks of the portfolio

def pflio_short(DF,m,x):
    df = DF.copy()
    portfolio = []
    monthly_ret_short = [0]
    for i in range(1,len(df)):
        if len(portfolio) > 0:
            monthly_ret_short.append(df[portfolio].iloc[i,:].mean())
            bad_stocks = df[portfolio].iloc[i,:].sort_values(ascending=False)[:x].index.values.tolist()
            portfolio = [t for t in portfolio if t not in bad_stocks]
        fill = m - len(portfolio)
        new_picks = df.iloc[i,:].sort_values(ascending=True)[:fill].index.values.tolist()
        new_picks = [t for t in new_picks if t not in portfolio]
        portfolio = portfolio + new_picks
    monthly_ret_df_short = pd.DataFrame(np.array(monthly_ret_short),columns=["mon_ret"])
    monthly_ret_df_short = -monthly_ret_df_short
    return monthly_ret_df_short

In [ ]:
# It is the combination of the Long only and Short only strategies 
# Long 10 stocks and Short 5
def pflio_long_short(DF, m, x):
        long_short = pd.DataFrame()
        long_short = 0.5 * pflio_long(return_df, m, x) + 0.5* (pflio_short(return_df, m-5, x-1))
        return long_short

### Backtesting strategy on F-F datasets

In [ ]:
#Strategy Long only sector rotational - Using monthly returns and rebalancing to handle better trends and transactions costs
strategy1 = pflio_long(df_industries_monthly_returns, 10, 5)
strategy1

In [ ]:
strategyret1 = 1 * (1 + strategy1).cumprod() # Index base = 1
strategyret1

In [ ]:
strategyret1.plot(figsize=(20, 10))
plt.title('Strategy 1 Index')

### Strategy 2

In [ ]:
#Strategy Long only sector rotational - Using monthly returns and rebalancing to handle better trends and transactions costs
strategy2 = pflio_long(df_industries_monthly_returns, 15, 5)
strategy2

In [ ]:
strategyret2 = 1 * (1 + strategy2).cumprod() # Index base = 1
strategyret2

In [ ]:
strategyret2.plot(figsize=(20, 10))
plt.title('Strategy 2 Index')

### Strategy 3 - Changing the start date

In [ ]:
start_date = '1995-01-01'
end_date = '2022-06-30'

In [ ]:
df = industries.copy(deep=True)
df

In [ ]:
df.reset_index(inplace = True)
df

In [ ]:
# Create a mask with conditional dates returning booleans
mask = (df['date'] > start_date) & (df['date'] <= end_date)
mask

In [ ]:
# Using previous mask to get a new dataframe with the right dates
dailyreturns_newdates = df.loc[mask]
dailyreturns_newdates

In [ ]:
dailyreturns_newdates.set_index("date", inplace=True)
dailyreturns_newdates

In [ ]:
# Plot the daily returns based on new dates
# Volatility clustering during crisis dates such as dot com bubble, 2008 crisis, sovereign debt crisis and covid
fig = px.line(dailyreturns_newdates)
fig.show()

In [ ]:
# Calculate the cumulative daily returns
df_cum_daily_returns_industries_newdate = 100*(1 + dailyreturns_newdates).cumprod() # index level start = 100
#df_cum_daily_returns = df_cum_daily_returns.reset_index()
df_cum_daily_returns_industries_newdate

In [ ]:
index_df_cum_daily_returns_industries_newdate = df_cum_daily_returns_industries_newdate
fig = px.line(index_df_cum_daily_returns_industries_newdate)
fig.show()

In [ ]:
# Aggregation by monthly returns with new dates
dailyreturns_newdates.index = pd.to_datetime(dailyreturns_newdates.index)
df_industries_monthly_returns_newdate = dailyreturns_newdates.resample('1M').agg(lambda x: (x + 1).prod() - 1)
df_industries_monthly_returns_newdate

In [ ]:
df_industries_monthly_returns_newdate.describe()

In [ ]:
#Strategy Long only sector rotational - Using monthly returns and rebalancing to handle better trends and transactions costs
strategy3 = pflio_long(df_industries_monthly_returns_newdate, 10, 5)
strategy3

In [ ]:
strategyret3 = 100 * (1 + strategy3).cumprod() # Index base = 1
strategyret3

In [ ]:
strategyret3.plot(figsize=(20, 10))
plt.title('Strategy 3 Index')

### Compare with benchmark : New date for Mkt-rf

In [ ]:
df_factors_newdate = factors.copy(deep=True)
df_factors_newdate

In [ ]:
df_factors_newdate.reset_index(inplace = True)
df_factors_newdate

In [ ]:
mask = (df_factors_newdate['date'] > start_date) & (df_factors_newdate['date'] <= end_date)
mask

In [ ]:
ff_dailyreturns_newdates = df_factors_newdate.loc[mask]
ff_dailyreturns_newdates

In [ ]:
ff_dailyreturns_newdates.set_index("date", inplace=True)
ff_dailyreturns_newdates

In [ ]:
fig = px.line(ff_dailyreturns_newdates)
fig.show()

In [ ]:
# Calculate the cumulative daily returns
df_cum_daily_returns_ff_newdate = 100*(1 + ff_dailyreturns_newdates).cumprod() # index level start = 100
#df_cum_daily_returns = df_cum_daily_returns.reset_index()
df_cum_daily_returns_ff_newdate


In [ ]:
index_df_cum_daily_returns_ff_newdate = df_cum_daily_returns_ff_newdate
fig = px.line(index_df_cum_daily_returns_ff_newdate)
fig.show()

In [ ]:
ff_dailyreturns_newdates.index = pd.to_datetime(ff_dailyreturns_newdates.index)
df_ff_monthly_returns_newdate = ff_dailyreturns_newdates.resample('1M').agg(lambda x: (x + 1).prod() - 1)
df_ff_monthly_returns_newdate

In [ ]:
df_ff_monthly_returns_newdate.describe()

In [ ]:
# We use the CAC40 as benchmark
start = '1995-01-31'
end = '2022-05-31'
cac40 = pd.DataFrame()
cac40 = yf.download("^FCHI",start,end,interval='1mo')
cac40["mon_ret"] = cac40["Adj Close"].pct_change()

### KPI functions

In [ ]:
def CAGR(DF):
    # function to calculate the Cumulative Annual Growth Rate of a strategy
    df = DF.copy()
    df["cum_return"] = (1 + df["mon_ret"]).cumprod()
    n = len(df)/12
    CAGR = (df["cum_return"].tolist()[-1])**(1/n) - 1
    return CAGR

In [ ]:
def volatility(DF):
    # function to calculate annualized volatility of a trading strategy"
    df = DF.copy()
    vol = df["mon_ret"].std() * np.sqrt(12)
    return vol

In [ ]:
def sharpe(DF,rf):
    # function to calculate sharpe ratio ; rf is the risk free rate"
    df = DF.copy()
    sr = (CAGR(df) - rf)/volatility(df)
    return sr

In [ ]:
def max_dd(DF):
    # function to calculate max drawdown
    df = DF.copy()
    df["cum_return"] = (1 + df["mon_ret"]).cumprod()
    df["cum_roll_max"] = df["cum_return"].cummax()
    df["drawdown"] = df["cum_roll_max"] - df["cum_return"]
    df["drawdown_pct"] = df["drawdown"]/df["cum_roll_max"]
    max_dd = df["drawdown_pct"].max()
    return max_dd

### Performance Statistics on full dataset since 1930

In [ ]:
print ('CAGR of Long only strategy = ' + str(round(CAGR(pflio_long(df_industries_monthly_returns, 10, 5))*100, 2)) + '%' )
print ('CAGR of CAC40 = ' + str(round(CAGR(cac40)*100, 2)) + '%' )

In [ ]:
print ('Vol of Long only strategy = ' + str(round(volatility(pflio_long(df_industries_monthly_returns, 10, 5))*100, 2)) + '%' )
print ('Vol of CAC40 = ' + str(round(volatility(cac40)*100, 2)) + '%' )

In [ ]:
rf = 0.012532
print ('Sharpe ratio of Long only strategy = ' + str(round(sharpe(pflio_long(df_industries_monthly_returns, 10, 5), rf), 2))  )
print ('Sharpe ratio of CAC40 = ' + str(round(sharpe(cac40, rf), 2)) )

In [ ]:
print ('Max Drawdown of Long only strategy = ' + str(round(max_dd(pflio_long(df_industries_monthly_returns, 10, 5))*100, 2)) + '%')
print ('Max Drawdown of CAC40 = ' + str(round(max_dd(cac40)*100, 2)) + '%')

In [ ]:
#Computation of the matrix of correlation for two decades
matrix_correlation = df_industries_monthly_returns.corr()
heatmap = sns.heatmap(data= matrix_correlation, annot= False)
heatmap

### New date stats performance Statistics 


In [ ]:
# Using new date
print ('CAGR of Long only strategy = ' + str(round(CAGR(pflio_long(df_industries_monthly_returns_newdate, 10, 5))*100, 2)) + '%' )
print ('CAGR of CAC40 = ' + str(round(CAGR(cac40)*100, 2)) + '%' )
ffretcagr = df_ff_monthly_returns_newdate.copy()
ffretcagr["cum_return"] = (1 + ffretcagr["Mkt-RF"]).cumprod()
n = len(ffretcagr)/12
ffCAGR = (ffretcagr["cum_return"].tolist()[-1])**(1/n) - 1
print ('CAGR of Market excess return F-F = ' + str(round(ffCAGR*100, 2)) + '%' )

In [ ]:
print ('Vol of Long only strategy = ' + str(round(volatility(pflio_long(df_industries_monthly_returns_newdate, 10, 5))*100, 2)) + '%' )
print ('Vol of CAC40 = ' + str(round(volatility(cac40)*100, 2)) + '%' )
ffvol = df_ff_monthly_returns_newdate.copy()
ff_vol = ffvol["Mkt-RF"].std() * np.sqrt(12)
print ('Vol of Market excess return F-F = ' + str(round(ff_vol*100, 2)) + '%' )

In [ ]:
rf = 0.00003*100
print ('Sharpe ratio of Long only strategy = ' + str(round(sharpe(pflio_long(df_industries_monthly_returns_newdate, 10, 5), rf), 2))  )
print ('Sharpe ratio of CAC40 = ' + str(round(sharpe(cac40, rf), 2)) )
ffsharpe = df_ff_monthly_returns_newdate.copy()
ff_sr = (ffCAGR - rf)/ff_vol
print ('Sharpe ratio of Market excess return F-F = ' + str(round(ff_sr, 2)) )

In [ ]:
print ('Max Drawdown of Long only strategy = ' + str(round(max_dd(pflio_long(df_industries_monthly_returns_newdate, 10, 5))*100, 2)) + '%')
print ('Max Drawdown of CAC40 = ' + str(round(max_dd(cac40)*100, 2)) + '%')
ffmdd = df_ff_monthly_returns_newdate.copy()
ffmdd["cum_return"] = (1 + ffmdd["Mkt-RF"]).cumprod()
ffmdd["cum_roll_max"] = ffmdd["cum_return"].cummax()
ffmdd["drawdown"] = ffmdd["cum_roll_max"] - ffmdd["cum_return"]
ffmdd["drawdown_pct"] = ffmdd["drawdown"]/ffmdd["cum_roll_max"]
max_dd = ffmdd["drawdown_pct"].max()
print ('Max Drawdown of Market excess return F-F = ' + str(round(max_dd*100, 2)) + '%')

In [ ]:
# Histogram Monthly strategy returns from new dates
strategy1.hist(bins = 100)

In [ ]:
# Histogram Monthly strategy returns from new dates
strategy2.hist(bins = 100)

In [ ]:
# Histogram Monthly strategy returns from new dates
strategy3.hist(bins = 100)

In [ ]:
# Histogram Monthly excess returns from new dates
df_ff_monthly_returns_newdate["Mkt-RF"].hist(bins = 100)

### Using PyPortfolioOpt library package for optimisation problem

In [ ]:
# https://pyportfolioopt.readthedocs.io/en/latest/UserGuide.html
# Read in price data
pd.DataFrame(df_cum_daily_returns_industries_newdate)
df_cum_daily_returns_industries_newdate.to_csv('industries_newdate.csv')
df = pd.read_csv("industries_newdate.csv", parse_dates=True, index_col="date")

# Calculate expected returns and sample covariance
mu = expected_returns.mean_historical_return(df)
S = risk_models.sample_cov(df, frequency =12)


# Optimize for maximal Sharpe ratio
ef = EfficientFrontier(mu, S)
weights = ef.max_sharpe()
ef.portfolio_performance(verbose=True)

In [ ]:
S

In [ ]:
mu

In [ ]:
weights

In [ ]:
plotting.plot_covariance(S, plot_correlation=True);

### Part 2

In [ ]:
crypto = pd.read_csv("crypto.csv", sep=",")
crypto["timestamp"] = pd.to_datetime(crypto["timestamp"])
crypto.set_index("timestamp", inplace=True)
crypto

In [ ]:
crypto = pd.read_csv("crypto.csv", sep=",", index_col="timestamp",parse_dates=True, usecols=["timestamp", "name", "close"])
crypto
#dfSPY = pd.read_csv('data/SPY.csv', index_col="Date", parse_dates=True, usecols=["Date", "Adj Close"], na_values=['nan'])

In [ ]:
datacrypto = crypto.reset_index().pivot("timestamp","name","close").reset_index().set_index("timestamp")
datacrypto

In [ ]:
#datacrypto.to_csv("datacrypto.csv")

In [ ]:
crypto.reset_index().pivot("timestamp","name","close").reset_index().set_index("timestamp").plot(logy=True)

In [ ]:
log_returns_crypto = np.log(datacrypto / datacrypto.shift(1))
log_returns_crypto

In [ ]:
# Replace NaN value by 0, because sometimes all indices are not open (depend of the local public holidays)
log_returns_crypto = log_returns_crypto.replace(np.nan, 0)
log_returns_crypto

In [ ]:
#Computation of the matrix of correlation
matrix_correlation = log_returns_crypto.corr()
sns.heatmap(data= matrix_correlation, annot= True)

In [ ]:
df_ret_crypto = crypto.reset_index().pivot("timestamp","name","close").pct_change().reset_index().set_index("timestamp")
df_ret_crypto
df_ret_crypto

In [ ]:
#df_ret_crypto.to_csv("df_ret_crypto.csv")

In [ ]:
df_daily_returns_crypto = df_ret_crypto
fig = px.line(df_daily_returns_crypto)
fig.show()

In [ ]:
# Calculate the cumulative daily returns
df_cum_daily_returns_crypto = 100*(1 + df_ret_crypto).cumprod() # index level start = 100
#df_cum_daily_returns = df_cum_daily_returns.reset_index()
df_cum_daily_returns_crypto

In [ ]:
index_df_cum_daily_returns_crypto = df_cum_daily_returns_crypto
fig = px.line(index_df_cum_daily_returns_crypto)
fig.show()

In [ ]:
df_cum_daily_returns_crypto.describe()

In [ ]:
unique_arr = crypto["name"].unique()
unique_arr

In [ ]:
len(unique_arr)

In [ ]:
mask = (crypto["name"] == "Bitcoin")
mask


In [ ]:
df = crypto.loc[mask]
df

In [ ]:
#Strategy Long only crypto rotational - Using monthly returns and rebalancing to handle better trends and transactions costs
cryptolongptf1 = pflio_long(df_industries_monthly_returns, 10, 5)
cryptolongptf1

In [ ]:
# Histogram Monthly strategy returns from new dates
df_ret_crypto["Bitcoin"].hist(bins = 100)

In [ ]:
# Histogram Monthly strategy returns from new dates
df_ret_crypto["XRP"].hist(bins = 100)

### Marketcap and Volume

In [ ]:
volume = pd.read_csv("crypto.csv", sep=",", index_col="timestamp",parse_dates=True, usecols=["timestamp", "name", "volume"])
volume

In [ ]:
market_cap = pd.read_csv("crypto.csv", sep=",", index_col="timestamp",parse_dates=True, usecols=["timestamp", "name", "market_cap"])
market_cap

In [ ]:
volume.reset_index().pivot("timestamp","name","volume").reset_index().set_index("timestamp").plot(logy=True)

In [ ]:
market_cap.reset_index().pivot("timestamp","name","market_cap").reset_index().set_index("timestamp").plot(logy=True)

In [ ]:
df_ret_crypto_volume = volume.reset_index().pivot("timestamp","name","volume").pct_change().reset_index().set_index("timestamp")
df_ret_crypto_volume

In [ ]:
df_ret_crypto_market_cap = market_cap.reset_index().pivot("timestamp","name","market_cap").pct_change().reset_index().set_index("timestamp")
df_ret_crypto_market_cap

In [ ]:
df_ret_crypto_volume.describe()

In [ ]:
df_ret_crypto_market_cap.describe()

In [ ]:
# Histogram Monthly strategy returns from new dates
df_ret_crypto_market_cap["Bitcoin"].hist(bins = 100)

In [ ]:
# Histogram daily strategy returns from new dates
df_ret_crypto_market_cap["XRP"].hist(bins = 100)